# Project 5

#### Description

Suppose we are writing an application that uses exceptions and we want our exception messages (and type) to be very consistent, as well as provide some way to easily list out all the possible exceptions used in our app.

Although there are many other approaches to doing this (as with any problem), let's use enumerations specifically to implement this functionality.

What we want is a mechanism whereby we can raise an exception this way:

```
AppException.Timeout.throw()
```
which will raise a custom exception `ConnectionException('100 - Timeout connecting to resource')`

And something like this as well:
```
AppException.NotAnInteger.throw()
```
which will raise a `ValueError('200 - Value is not an integer')`

This means our exception will need to contain the exception key (such as `Timeout` or `NotAnInteger`) as well as the exception class we want to raise, and the default message itself. We also want to have consistent error codes (integer values) for each exception.

We'll need to implement a `throw` method (we can't use the reserved name `raise`) that will raise the exception with the default message. In addition we'd like to be able to override the default message with a custom one if we prefer:
```
AppException.Timeout.throw('Timeout connecting to database')
```

We'll also need to implement some properties for the exception code, class (type), and message.

#### Main Solution

First let's create a few custom exceptions that we can use, but of course we can also use all the builtin exceptions too.

In [2]:
class GenericException(Exception):
    pass

class Timeout(Exception):
    pass

In [3]:
from enum import Enum, unique

class AppException(Enum):
    Generic = (100, GenericException, 'Application exception.')
    Timeout = (101, Timeout, 'Timeout connecting to resource.')
    NotAnInteger = (200, ValueError, 'Value must be an integer.')
    NotAList = (201, ValueError, 'Value must be a list.')
    
    def __new__(cls, ex_code, ex_class, ex_message):
        # create a new instance of cls
        member = object.__new__(cls)
        
        # set up instance attributes
        member._value_ = ex_code
        member.exception = ex_class
        member.message = ex_message
        return member

So this is a good start. We can use our enum this way:

In [4]:
AppException.Timeout.value, AppException.Timeout.message, AppException.Timeout.exception

(101, 'Timeout connecting to resource.', __main__.Timeout)

Since we passed the exception type which is a callable into each enum, we can raise the exception simply by calling it:

In [14]:
raise AppException.Timeout.exception(f'\n\tRaising Error code: {AppException.Timeout.value}.\n\tMessage: {AppException.Timeout.message}')

Timeout: 
	Raising Error code: 101.
	Message: Timeout connecting to resource.

This is not what we want in two ways:
1. Firstly, we want the ability to do `AppException.Timeout.code` which should give us the same thing as `AppException.Timeout.value`. We will resolve this with a `property` which will act like an alias.
2. The formatting for when we raise an exception is verbose. We want it to autoformat but allow us to override if we want. We will resolve this with a method `.throw()` which will contain the formatting implementation and overridability.


In [17]:
class AppException(Enum):
    Generic = (100, GenericException, 'Application exception.')
    Timeout = (101, Timeout, 'Timeout connecting to resource.')
    NotAnInteger = (200, ValueError, 'Value must be an integer.')
    NotAList = (201, ValueError, 'Value must be a list.')
    
    def __new__(cls, ex_code, ex_class, ex_message):
        # create a new instance of cls
        member = object.__new__(cls)
        
        # set up instance attributes
        member._value_ = ex_code
        member.exception = ex_class
        member.message = ex_message
        return member
    
    @property
    def code(self):
        return self.value
    
    def throw(self, message=None):
        message = message or self.message
        raise self.exception(f'\n\tRaising Error code: {AppException.Timeout.value}.\n\tMessage: {AppException.Timeout.message}')

In [18]:
raise AppException.Timeout.throw()

Timeout: 
	Raising Error code: 101.
	Message: Timeout connecting to resource.

In [20]:
raise AppException.Timeout.throw('Timeout connecting to the database')

Timeout: 
	Raising Error code: 101.
	Message: Timeout connecting to resource.

If a user wants to know all the exceptions we have, we can simply list them out:

In [21]:
list(AppException)

[<AppException.Generic: 100>,
 <AppException.Timeout: 101>,
 <AppException.NotAnInteger: 200>,
 <AppException.NotAList: 201>]